# HW.1
## Optimization and Analytics 23/24-S1

FELIX GRANDNER, MIN JEGAL


In [1]:
import pandas as pd

#from google.colab import drive

#drive.mount('/content/drive')
#path_to_folder = '/content/drive/My Drive/Colab Notebooks/Optimization/Lab1/'


# 1. Introduction
Our project is to  'Minimize the total cost of oil transportation for the EU countries'.

To conduct our project we crawled the data about popluation from each country and the amount of oil the country has from the wikipedia. $^1)$ <br>
The 'distance matrix.csv' is the data about the distance between the countries, 'oil_countries' contains the information of the amount of oil each country has, and the 'eu_countries' is the data of the population of each country. Due to the complexity to get the data of the exact number of the consumption of oil, we modified the data by 'percent of toatl EU-27 area' * 100 and rounded the number.



# ------------------ #
$^1)$

1. https://en.wikipedia.org/wiki/Demographics_of_the_European_Union <br>
2. https://en.wikipedia.org/wiki/List_of_countries_by_oil_production

# I. The First Part
The goal of our first model is to determine the optimal distribution of oil imports among EU countries from various oil-producing countries, considering both economic and environmental factors. The model aims to minimize the total cost of transportation, which includes the direct cost associated with the distance of transportation and the environmental impact in terms of carbon emissions. Moreover our model consider variable transportation capacities, reflecting real-world scenarios where transportation resources might fluctuate.

To calculate the distances between the countries we gathered the geocoordinates of each capital city and computed the distance on a globe using the haversine distance function.

In [2]:
import csv

# Initialize an empty list to store the data
data = []

# Specify the path to the CSV file
csv_file = 'distances_matrix.csv'


# Read the data from the CSV file and exclude the first entry in each row
try:
    with open(csv_file, 'r', newline='', encoding='iso-8859-1') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)  # Read the header row (if it exists)

        for row in reader:
            # Exclude the first entry in each row using slicing [1:]
            data.append(row[1:])

    # Print the loaded data
    print("Loaded Data (excluding the first entry in each row):")
    for row in data:
        print(row)
except UnicodeDecodeError as e:
    print(f"UnicodeDecodeError: {e}")
except FileNotFoundError as e:
    print(f"File not found: {e}")
except Exception as e:
    print(f"An error occurred: {e}")


Loaded Data (excluding the first entry in each row):
['6709.0', '6164.0', '7217.0', '6089.0', '7178.0', '7977.0', '6191.0', '6217.0', '8254.0', '6899.0', '6636.0', '5735.0', '7336.0', '7124.0', '7912.0', '6511.0', '6930.0', '7174.0', '5443.0', '7233.0', '7270.0', '7122.0', '7075.0', '6963.0', '9113.0', '6386.0', '7721.0']
['4168.0', '4681.0', '3673.0', '4960.0', '3751.0', '2877.0', '4658.0', '4624.0', '2621.0', '3955.0', '4448.0', '5421.0', '3513.0', '3722.0', '2928.0', '4418.0', '4286.0', '3674.0', '5398.0', '3607.0', '3773.0', '3719.0', '4017.0', '4223.0', '1731.0', '4454.0', '3312.0']
['1610.0', '2486.0', '2375.0', '3439.0', '1150.0', '1498.0', '2146.0', '2252.0', '2231.0', '1667.0', '1227.0', '3906.0', '1568.0', '1670.0', '1777.0', '1560.0', '892.0', '1631.0', '2793.0', '1868.0', '790.0', '1932.0', '840.0', '866.0', '2311.0', '2207.0', '2815.0']
['6126.0', '5647.0', '6728.0', '5690.0', '6576.0', '7411.0', '5631.0', '5674.0', '7745.0', '6333.0', '5993.0', '5384.0', '6774.0', '6566.0

Thus according to the goal we set, we could define the model as the follwoing


### Optimization Model : EU oil import problem, considering Ecological Factor

#### Sets:
- $I$: Set of oil-producing countries.
- $J$: Set of EU countries.

#### Parameters:
- $\text{Cap}_i$: Oil production capacity for country $i \in I$.
- $\text{Dem}_j$: Oil demand for EU country $j \in J$.
- $\text{Distance}_{i,j}$: Distance between oil-producing country $i$ and EU country $j$.
- $\text{carbon emission coefficient}$: Coefficient for carbon emissions that describes the rate at which a given activity releases greenhouse gases (GHGs) into the atmosphere.

#### Variables:
- $x_{i,j}$: Quantity of oil transported from country $i$ to country $j$.
- $\text{TransCap}_i$: Variable transportation capacities for country $i$.

#### Objective Function:
Minimize the total cost of transportation and environmental impact.
$$ \min \sum_{i \in I, j \in J} (0.01 \times \text{Distance}_{i,j} \times x_{i,j} + \text{carbon_emission_coefficient} \times \text{Distance}_{i,j} \times x_{i,j}) $$

#### Constraints:
##### Capacity Constraint:
We simulate a embargo on russian oil by setting the capacity to 300.
For non-Russian countries:
$$ \sum_{j \in J} x_{i,j} \leq \text{Cap}_i \, \forall i \in I \setminus \{\text{"Russia"}\} $$
For Russia:
$$ \sum_{j \in J} x_{\text{"Russia"},j} \leq 300 $$

##### Demand Constraint:
$$ \sum_{i \in I} x_{i,j} \geq \text{Dem}_j \, \forall j \in J $$

##### Variable Capacity Adjustment:
$$ \text{TransCap}_i = 0.75 \times \text{Cap}_i \, \forall i \in I $$


In [3]:
#pip install Pyomo

In [4]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

import csv

model = ConcreteModel('EU oil import problem')


oil_countries = {
    "United States": 6000,
    "Saudi Arabia": 5300,
    "Russia": 5100,
    "Canada": 2200,
    "Iraq": 2200,
    "United Arab Emirates": 1700,
    "Iran": 1600,
    "Norway": 1500,
    "Brazil": 1500,
    "Kuwait": 1400,
    "Kazakhstan": 1300,
    "Mexico": 900,
    "Algeria": 700,
    "Qatar": 600,
    "Nigeria": 600
}

for country, value in oil_countries.items():
    oil_countries[country] = 3*value/4


eu_countries = {
    "Germany": 840,
    "France": 680,
    "Italy": 600,
    "Spain": 470,
    "Poland": 380,
    "Romania": 190,
    "Netherlands": 180,
    "Belgium": 115,
    "Greece": 110,
    "Czech Republic": 110,
    "Sweden": 105,
    "Portugal": 103,
    "Hungary": 98,
    "Austria": 89,
    "Bulgaria": 70,
    "Denmark": 58,
    "Finland": 56,
    "Slovakia": 55,
    "Ireland": 49,
    "Croatia": 40,
    "Lithuania": 28,
    "Slovenia": 21,
    "Latvia": 19,
    "Estonia": 13,
    "Cyprus": 10,
    "Luxembourg": 6,
    "Malta": 5
}

model.i = Set(initialize=oil_countries.keys(), doc='oil producing countries')
model.j = Set(initialize=eu_countries.keys(), doc='eu countries')

model.Cap = Param(model.i, initialize=oil_countries, doc='oil capacity')
model.Dem = Param(model.j, initialize=eu_countries, doc='oil demand')

distance_data = {}
for i, origin in enumerate(model.i):
    for j, destination in enumerate(model.j):
        distance_data[(origin, destination)] = int(round(float(data[i][j])))

carbon_emission_coefficient = 0.005
model.Distance = Param(model.i, model.j, initialize=distance_data, doc='Distance in km')
model.TransCap = Var(model.i, within=NonNegativeReals, doc='Variable transportation capacities')



In [5]:
model.x = Var(model.i, model.j, doc="oil bought transported", within=NonNegativeReals)

def objective_rule(model):
  transport_cost = sum(0.01*model.Distance[i, j]*model.x[i, j] for i in model.i for j in model.j)
  environmental_impact = sum(carbon_emission_coefficient * model.Distance[i, j] * model.x[i, j] for i in model.i for j in model.j)
  return transport_cost + environmental_impact

model.objective = Objective(rule=objective_rule, sense=minimize, doc="oil cost")

def Capacity(model, i):
    if(i == "Russia"):
        return sum(model.x[i,j] for j in model.j) <= 300
    else:
        return sum(model.x[i,j] for j in model.j) <= model.Cap[i]

model.Capacity = Constraint(model.i, rule=Capacity, doc="capacity at oil country + Russia sanctions")

def Demand(model, j):
    return sum(model.x[i,j] for i in model.i) >= model.Dem[j]

model.Demand = Constraint(model.j, rule=Demand, doc="consumption in eu country")


for country in model.i:
    model.TransCap[country] = 0.75 * model.Cap[country]


The problem is structured as a linear optimization problem, with an aim to minimize costs subject to various linear constraints (like capacity and demand constraints), we used 'glpk' as a solver

In [6]:
#!apt-get install -y -qq glpk-utils

In [7]:
model.dual = Suffix(direction=Suffix.IMPORT)


In [8]:
Solver = SolverFactory('glpk')
Results = Solver.solve(model)

# Display solution
#display(model)

import pandas as pd

table_results = pd.DataFrame()

for i in model.i:
    for j in model.j:
        table_results.loc[i,j] = model.x[i,j]()

# Save pandas tabel into excel file.


print(table_results)
model.objective()

                      Germany  France  Italy  Spain  Poland  Romania  \
United States             0.0     0.0    0.0    0.0     0.0      0.0   
Saudi Arabia              0.0     0.0    0.0    0.0     0.0      0.0   
Russia                  177.0     0.0    0.0    0.0     0.0      0.0   
Canada                    0.0     0.0    0.0    0.0     0.0      0.0   
Iraq                    246.0     0.0  600.0    0.0     0.0    190.0   
United Arab Emirates      0.0     0.0    0.0    0.0     0.0      0.0   
Iran                    417.0     0.0    0.0    0.0   380.0      0.0   
Norway                    0.0   625.0    0.0    0.0     0.0      0.0   
Brazil                    0.0     0.0    0.0    0.0     0.0      0.0   
Kuwait                    0.0     0.0    0.0    0.0     0.0      0.0   
Kazakhstan                0.0     0.0    0.0    0.0     0.0      0.0   
Mexico                    0.0     0.0    0.0    0.0     0.0      0.0   
Algeria                   0.0    55.0    0.0  470.0     0.0     

143680.695

### Optimized solution

In the table above we see the optimized solution for every EU state

In [9]:
# to include marginal prices
model.dual.display()


dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key                            : Value
                 Capacity[Algeria] :            -40.56
                  Capacity[Brazil] :               0.0
                  Capacity[Canada] :               0.0
                    Capacity[Iran] :               0.0
                    Capacity[Iraq] : -3.69000000000001
              Capacity[Kazakhstan] :               0.0
                  Capacity[Kuwait] :               0.0
                  Capacity[Mexico] :               0.0
                 Capacity[Nigeria] :               0.0
                  Capacity[Norway] :            -40.62
                   Capacity[Qatar] :               0.0
                  Capacity[Russia] :           -28.455
            Capacity[Saudi Arabia] :               0.0
    Capacity[United Arab Emirates] :               0.0
           Capacity[United States] :               0.0
                   Demand[Austria] :              46.5
                   Dema

### Lagrange multipliers

The table above provides the exact lagrange multpliers for every constraint of our problem. We can observe that the most expensive demand at this moment is France. Adding 1 unit in the demand of France would result in a final solution increase of 60.735.
The same can be said for the capacities. Most capacities do not contribute to the optimal solution but adding 1 unit capacity in Norway or Algeria results in a change of around -40.5 in the optimal solution, which seems plausible because the are two of the nearest providers of oil which results in a low cost for the EU in our model.

# II. The Second Part

The objective of the model is to enhance the distribution of oil imports among EU countries with an emphasis on various constraints. This entails not only optimizing economic factors, such as minimizing transportation costs, but also adhering to geopolitical and logistical limitations, including production capacities and diversification requirements. Additional considerations, such as transatlantic trade specifics, are also taken into account. A proposed solution involves the construction of a new pipeline at a cost of 8000€, which has the potential to significantly reduce transportation costs for transatlantic nations. The optimization process will assess whether the investment in the pipeline is justified. As part of our strategy, we have implemented a policy that mandates each EU country to source oil from at least 5 different countries. To add a layer of decision-making complexity mirroring real-world choices in international trade and supply chain management, binary variables will be employed. Furthermore, a diversification policy for EU countries has been implemented through constraints that stipulate the importation of oil from a minimum number of different sources. This comprehensive approach aims to ensure a well-balanced and strategic distribution of oil imports within the specified constraints.

Thus according to the goal we set, we can define our model as following.
However this model can be a non-linear problem but for the consistency of using the same solver, we introduced the new auxiliary variable $z_i,_j$ and the linearization constranints.

### Updated Optimization Model - EU Oil Import Problem, considering Political Factors

#### Sets:
- $I$: Set of oil-producing countries.
- $J$: Set of EU countries.

#### Parameters:
- $\text{Cap}_i$: Oil production capacity for country $i \in I$.
- $\text{Dem}_j$: Oil demand for EU country $j \in J$.
- $\text{Distance}_{i,j}$: Distance between oil-producing country $i$ and EU country $j$.

#### Variables:
- $x2_{i,j}$: Quantity of oil transported from country $i$ to country $j$.
- $y$: Binary variable for a specific condition in the objective function.
- $y2_{i,j}$: Binary variable for diversification constraints.
- $z_{i,j}$: Auxiliary variable representing the linearized product of $x2_{i,j}$ and $y$.

#### Objective Function:
Minimize the total transportation cost and an additional cost factor, considering linearization of the product term:
$$ \min \sum_{i \in I, j \in J} 0.01 \times \text{Distance}_{i,j} \times x2_{i,j} + 8000 \times y - 0.005 \times \sum_{i \in \text{transatlantic}, j \in J} \text{Distance}_{i,j} \times z_{i,j} $$

#### Constraints:
- **Capacity Constraint:**
  - For Russia: $\sum_{j \in J} x2_{\text{"Russia"},j} \leq 300$
  - For other countries: $\sum_{j \in J} x2_{i,j} \leq \text{Cap}_i \, \forall i \in I$
- **Demand Constraint:**
  - $\sum_{i \in I} x2_{i,j} \geq \text{Dem}_j \, \forall j \in J$
- **Diversification Constraint:**
  - $\sum_{i \in I} y2_{i,j} \geq 5 \, \forall j \in J$
- **Minimum Usage Constraint:**
  - $x2_{i,j} \geq 0.1 \times \text{Dem}_j \times y2_{i,j} \, \forall i \in I, j \in J$
- **Binary Constraints:**
  - $x2_{i, j} \leq y2_{i, j} \times 10000$
  - $x2_{i, j} \geq y2_{i, j} / 10000$
- **Linearization Constraints:**
  - $z_{i, j} \leq x2_{i, j}$
  - $z_{i, j} \leq 10000 \times y$
  - $z_{i, j} \geq x2_{i, j} - 10000 \times (1 - y)$


In [10]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

import csv

model = ConcreteModel('EU oil import problem part2')


oil_countries = {
    "United States": 6000,
    "Saudi Arabia": 5300,
    "Russia": 5100,
    "Canada": 2200,
    "Iraq": 2200,
    "United Arab Emirates": 1700,
    "Iran": 1600,
    "Norway": 1500,
    "Brazil": 1500,
    "Kuwait": 1400,
    "Kazakhstan": 1300,
    "Mexico": 900,
    "Algeria": 700,
    "Qatar": 600,
    "Nigeria": 600
}

for country, value in oil_countries.items():
    oil_countries[country] = 3*value/4


eu_countries = {
    "Germany": 840,
    "France": 680,
    "Italy": 600,
    "Spain": 470,
    "Poland": 380,
    "Romania": 190,
    "Netherlands": 180,
    "Belgium": 115,
    "Greece": 110,
    "Czech Republic": 110,
    "Sweden": 105,
    "Portugal": 103,
    "Hungary": 98,
    "Austria": 89,
    "Bulgaria": 70,
    "Denmark": 58,
    "Finland": 56,
    "Slovakia": 55,
    "Ireland": 49,
    "Croatia": 40,
    "Lithuania": 28,
    "Slovenia": 21,
    "Latvia": 19,
    "Estonia": 13,
    "Cyprus": 10,
    "Luxembourg": 6,
    "Malta": 5
}

model.i = Set(initialize=oil_countries.keys(), doc='oil producing countries')
model.j = Set(initialize=eu_countries.keys(), doc='eu countries')

model.Cap = Param(model.i, initialize=oil_countries, doc='oil capacity' )
model.Dem = Param(model.j, initialize=eu_countries, doc='oil demand')

distance_data = {}
for i, origin in enumerate(model.i):
    for j, destination in enumerate(model.j):
        distance_data[(origin, destination)] = int(round(float(data[i][j])))


model.Distance = Param(model.i, model.j, initialize=distance_data, doc='Distance in km')


In [11]:
model.x2 = Var(model.i, model.j, doc="oil bought transported", within=NonNegativeReals)
model.y = Var(domain=Binary)
model.y2 = Var(model.i, model.j, domain=Binary)


###---------#### for non linearlity
model.z = Var(model.i, model.j, within=NonNegativeReals)

def objective_rule(model):
    transatlantic = ["United States", "Canada", "Mexico", "Brazil"]

    # First linear part: sum of transportation costs
    transportation_costs = sum(0.01 * model.Distance[i, j] * model.x2[i, j]
                               for i in model.i for j in model.j)

    # Second linear part: modified term with auxiliary variable
    modified_term = sum(model.Distance[i, j] * model.z[i, j]
                        for i in transatlantic for j in model.j)

    return transportation_costs + 8000 * model.y - 0.005 * modified_term


model.objective = Objective(rule=objective_rule, sense=minimize)


def Capacity(model, i):
    if(i == "Russia"):
        return sum(model.x2[i,j] for j in model.j) <= 300
    else:
        return sum(model.x2[i,j] for j in model.j) <= model.Cap[i]

model.Capacity = Constraint(model.i, rule=Capacity, doc="capacity at oil country")

def Demand(model, j):
    return sum(model.x2[i,j] for i in model.i) >= model.Dem[j]
model.Demand = Constraint(model.j, rule=Demand, doc="consumption in eu country")


def Diversify(model, j):
    return sum(model.y2[i,j] for i in model.i) >= 5 #  each EU country gets oil from at least 5 different countries
model.Diversify = Constraint(model.j, rule=Diversify, doc="diversify constraint")

def Diversify2(model, i, j):
    return model.x2[i,j] >= 0.1*model.Dem[j]*model.y2[i,j]
model.Diversify2 = Constraint(model.i, model.j, rule=Diversify2, doc="minimum usage")


def BinaryDefinition(model, i, j):
    return model.x2[i, j] <= model.y2[i, j] * 10000
model.Binary = Constraint(model.i, model.j, rule=BinaryDefinition, doc="binary constraint (1)")

def BinaryDefinition2(model, i, j):
    return model.x2[i, j] >= model.y2[i, j] / 10000
model.Binary2 = Constraint(model.i, model.j, rule=BinaryDefinition2, doc="binary constraint (2)")

###---------#### for non linearlity
def linearization_constraint1(model, i, j):
    return model.z[i, j] <= model.x2[i, j]

def linearization_constraint2(model, i, j):
    return model.z[i, j] <= 10000 * model.y

def linearization_constraint3(model, i, j):
    return model.z[i, j] >= model.x2[i, j] - 10000 * (1 - model.y)

model.Linearization1 = Constraint(model.i, model.j, rule=linearization_constraint1)
model.Linearization2 = Constraint(model.i, model.j, rule=linearization_constraint2)
model.Linearization3 = Constraint(model.i, model.j, rule=linearization_constraint3)




In [12]:
Solver = SolverFactory('glpk')
Results = Solver.solve(model)

#display(model)


import pandas as pd

table_results = pd.DataFrame()

for i in model.i:
    for j in model.j:
        table_results.loc[i,j] = model.x2[i,j]()


print(table_results)

print(model.y())
print(model.objective())


                      Germany  France  Italy  Spain  Poland  Romania  \
United States            84.0    68.0    0.0   47.0     0.0      0.0   
Saudi Arabia              0.0     0.0    0.0    0.0     0.0     19.0   
Russia                   84.0     0.0    0.0    0.0   122.9      0.0   
Canada                   84.0   408.0   60.0   47.0    38.0      0.0   
Iraq                     84.0    68.0  292.9    0.0    44.3    114.0   
United Arab Emirates      0.0     0.0    0.0    0.0     0.0      0.0   
Iran                      0.0     0.0   60.0    0.0    38.0     19.0   
Norway                  504.0    68.0    0.0    0.0   136.8      0.0   
Brazil                    0.0     0.0    0.0   47.0     0.0      0.0   
Kuwait                    0.0     0.0   60.0    0.0     0.0     19.0   
Kazakhstan                0.0     0.0    0.0    0.0     0.0      0.0   
Mexico                    0.0     0.0    0.0    0.0     0.0      0.0   
Algeria                   0.0    68.0  127.1  282.0     0.0     

### Optimal solution

In our case the new transatlantic pipeline gets built so lots of oil is now flowing from transatlantic states, in test we saw if we increase the cost of the pipeline only by 1000 units the pipeline would not be built and the optimal solution would not involve any transatlantic oil. Furthermore the diversification constraint is not letting countries rely solely on one country's oil production, still we can see the eu-countries purchasing the minimal amount (10%) necessary from not perfectly optimal suppliers.